In [ ]:
import random
import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

def generate_instance(nb_vertex, min_weight, max_weight):
  """
  Generate a random instance of a graph
  """
  graph = np.ones((nb_vertex, nb_vertex))*np.inf
  for i in range(nb_vertex - 1):
    for j in range(i+1, nb_vertex):
      if random.uniform(0, 1) < 0.5:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  for i,line in enumerate(graph):
    index_zeros = np.where(line == np.inf)[0]
    index_zeros = np.delete(index_zeros, np.where(index_zeros == i))
    for j in range(len(index_zeros) - (nb_vertex - 3)):
      index = random.choice(index_zeros)
      index_zeros = np.delete(index_zeros, np.where(index_zeros == index))
      weight = random.randint(min_weight, max_weight)
      graph[i, index] = weight
      graph[index, i] = weight
  return graph

def generate_trafic(nb_vertex, min_weight, max_weight):
  """
  Generate a random instance of a trafic
  """
  graph = np.zeros((nb_vertex, nb_vertex))
  for i in range(nb_vertex - 1):
    for j in range(i + 1, nb_vertex):
      if random.uniform(0, 1) < 0.2:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  return graph



graph = generate_instance(4,1,5)
trafic = generate_trafic(4,1,5)
print(graph)
print("graph",graph)
print("trafic",trafic)
"""G = nx.from_numpy_matrix(np.array(graph))  
nx.draw(G, with_labels=True) """

In [ ]:
import numpy as np
from dijkstar import Graph, find_path

complet_graph_list = []
points_path = {}

def convert_graph(graph):
  graph_dijkstra = Graph()

  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  
  return graph_dijkstra


def check_connection_between_two_nodes(graph, FirstNode, SecondNode):
  global complet_graph_list
  
  """
  Check if two nodes are connected
  """
  if (graph[FirstNode][SecondNode] != np.inf):
    complet_graph_list.append(graph[FirstNode][SecondNode])
  else:
    if FirstNode != SecondNode:

      converting_graph = convert_graph(graph)
      shortest_path = find_path(converting_graph, FirstNode, SecondNode)
      print(shortest_path)
      distance, path = shortest_path.total_cost, shortest_path.nodes
      complet_graph_list.append(int(distance))
      points_path[(FirstNode, SecondNode)] = path
    else:
      complet_graph_list.append(0)
    
for i in range(len(graph)):
  for j in range(len(graph)):
    check_connection_between_two_nodes(graph, i, j)

reshape_matrix = np.reshape(complet_graph_list, (len(graph),len(graph)))
print(reshape_matrix)

G = nx.from_numpy_matrix(np.array(reshape_matrix))  
nx.draw(G, with_labels=True)


In [ ]:

graph_dijkstra = Graph()
for i in range(len(graph)):
    for j in range(len(graph)):
        graph_dijkstra.add_edge(i, j,graph[i][j])

shortest_path = find_path(graph_dijkstra, 2, 3)

print(shortest_path.total_cost)

